### ztf_sim_introduction

This notebook illustrates basic use of the `ztf_sim` modules.

In [ ]:
# hack to get the path right
import sys
sys.path.append('..')

In [ ]:
import ztf_sim
from astropy.time import Time

Let's load the Fields object with the default field grid loaded:

In [ ]:
f = ztf_sim.fields.Fields()

The raw fieldid and coordinates are stored as a pandas Dataframe in the `.fields` attribute:

In [ ]:
f.fields

Now let's calcuate their altitude and azimuth at a specific time using the astropy.time.Time object:

In [ ]:
f.alt_az(Time.now())